In [1]:
import os
import json
import re
from tqdm import tqdm
import pickle
import itertools
import networkx as nx
from matplotlib import pyplot as plt
import networkx
import pandas as pd
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, EdgesAndLinkedNodes, NodesAndLinkedEdges, LabelSet
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from networkx.algorithms import community

In [3]:
path = './final_proj/'
files = sorted(os.listdir(path))
N = len(files)

In [ ]:
extracted_words = {}
for j in tqdm(range(N)):
    with open(path + files[j]) as json_file:
        data = json.load(json_file)
    num_blocks = len(data['result']['blocks'])
    extracted_words_set = set()
    
    for i in range(num_blocks):
        if data['result']['blocks'][i]['type'] == 'text' or data['result']['blocks'][i]['type'] == 'header':
            extracted_words_set = extracted_words_set.union(set(re.findall(r'«[\w]+»', data['result']['blocks'][i]['data']['text'])))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'"[\w]+"', data['result']['blocks'][i]['data']['text'])))
            #print(re.findall(r'[\s]?[A-Z][A-Za-z]+[\s]|[\s]?[А-Я][А-Яа-я]+[\s]', data['result']['blocks'][i]['data']['text']))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z][A-Za-z]*[\.][A-Z][A-Za-z]*|[А-Я][А-Яа-я]*[\.][А-Я][А-Яа-я]*', data['result']['blocks'][i]['data']['text'])))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z]+[^a-z]|[А-Я]+[^а-я]', data['result']['blocks'][i]['data']['text'])))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z][a-z]+|[А-Я][а-я]+', data['result']['blocks'][i]['data']['text'])))
            
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z][a-z]+[\s-][A-Z][a-z]+|[А-Я][а-я]+[\s-][А-Я][а-я]+', data['result']['blocks'][i]['data']['text'])))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z][a-z]+\s&\s[A-Z][a-z]+|[A-Z][a-z]+&[A-Z][a-z]+', data['result']['blocks'][i]['data']['text'])))

    extracted_words[files[j]] = [item.lower().strip('«.» ,') for item in extracted_words_set]
    #        print(data['result']['blocks'][i]['data']['text'].split())

In [12]:
with open('mapa_comp.pkl', 'rb') as f:
    mapa = pickle.load(f)

In [6]:
with open('companies_stopwords.txt', 'r') as f:
    stop_words_comp = f.read()
stop_words_comp = set(stop_words_comp.split('\n'))

In [963]:
with open('companies_habr_upd.txt', 'r') as f:
    companies_habr = f.read()
companies_habr = set(companies_habr.split('\n'))

In [964]:
with open('companies_vc_upd.txt', 'r') as f:
    companies_vc = f.read()
companies_vc = set(companies_vc.split('\n'))

In [965]:
with open('companies_add.txt', 'r') as f:
    companies_add = f.read()
companies_add = set(companies_add.split('\n'))

In [966]:
companies = companies_habr.union(companies_vc).union(companies_add) - stop_words_comp

In [967]:
stop_w2 = set(['tj'])

In [968]:
extracted_companies = {}
for key in tqdm(extracted_words.keys()):
    for word in extracted_words[key]:
        if (word in companies) and (not word in stop_w2) and ((not word in mapa.keys()) or  ((word in mapa.keys()) and (not mapa[word] in stop_w2))):
            if not key in extracted_companies.keys():
                extracted_companies[key] = []
            if word in mapa.keys():
                extracted_companies[key].append(mapa[word])
            else:
                extracted_companies[key].append(word)

100%|███████████████████████████████| 298694/298694 [00:01<00:00, 219688.04it/s]


In [969]:
for key in extracted_companies.keys():
    extracted_companies[key] = list(set(extracted_companies[key]))

In [988]:
#pd.set_option('display.max_rows', 2500)
extracted_companies_ = {}
for key in extracted_companies.keys():
    extracted_companies_[key] = ';'.join(extracted_companies[key])

In [972]:
#df['companies'] = df['companies'].str.split(';')
#df = df.explode('companies')
#df.groupby('companies').agg('count').sort_values('index').reset_index()

In [973]:
edges = []
for key in extracted_companies.keys():
    if len(extracted_companies[key]) > 1:
        for element in itertools.product(*[extracted_companies[key], extracted_companies[key]]):
            edges.append(element)

In [974]:
edges_ = {}
for item in tqdm(edges):
    if item[0] != item[1]:
        if (not (item[0], item[1]) in edges_.keys()) and (not (item[1], item[0]) in edges_.keys()):
            edges_[item] = 0.0
        else:
            if (item[0], item[1]) in edges_.keys():
                edges_[item] += 1.0
            elif (item[1], item[0]) in edges_.keys():
                edges_[(item[1], item[0])] += 1.0

100%|███████████████████████████████| 245112/245112 [00:00<00:00, 590759.33it/s]


In [975]:
G = nx.Graph()
for key in edges_.keys():
    if edges_[key] > 1:
        G.add_weighted_edges_from([(key[0], key[1], edges_[key])])

In [976]:
degrees = dict(networkx.degree(G))
networkx.set_node_attributes(G, name='degree', values=degrees)

In [977]:
number_to_adjust_by = 3
adjusted_node_size = dict([(node, 0.1*(degree+number_to_adjust_by)) for node, degree in networkx.degree(G)])
networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

In [978]:
from networkx.algorithms import community
communities = community.greedy_modularity_communities(G, weight='weight', resolution=1.5)

In [980]:
len(communities)

28

In [981]:
# Create empty dictionaries
modularity_class = {}
modularity_color = {}
colors = (
'#3288bd',
'#66c2a5',
'#abdda4',
'#e6f598',
'#fee08b',
'#fdae61',
'#f46d43',
'#d53e4f',   
'#8c2d04',
'#8B3A62',
'#EE2C2C',
    
'#BFEFFF',
'#68838B',
'#FFA07A',
'#E066FF',
'#5D478B',
'#48D1CC',
'#FFE4E1',
    '#000080',
    '#FFEFD5',
    '#308014',
    
    '#7D9EC0',
    '#C67171',
    '#8B7B8B',
    '#8B8B00',
    '#838B8B',
    '#8B8378',
    '#8A360F',
    '#C1FFC1',
    
)
#Loop through each community in the network
for community_number, community in enumerate(communities):
    #For each member of the community, add their community number and a distinct color
    for name in community: 
        modularity_class[name] = community_number
        modularity_color[name] = colors[community_number]

In [982]:
networkx.set_node_attributes(G, modularity_class, 'modularity_class')
networkx.set_node_attributes(G, modularity_color, 'modularity_color')

In [956]:
#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'
edge_color = 'gray'

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'Companies graph'


#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Companies", "@index"),
        ("Degree", "@degree"),
         ("Modularity Class", "@modularity_class"),
      #  ("Modularity Color", "$color[swatch]:modularity_color"),
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)



#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.3, line_color=edge_color, line_width=1)
#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(network_graph)

#Add Labels
x, y = zip(*network_graph.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
#save(plot, filename=f"{title}.html")

In [983]:
node_list = list(G.nodes)

## Find close companies via Jaccard score of neighbours

In [984]:
from collections import defaultdict
attr_relevant_nodes = defaultdict(list)
for i in tqdm(range(len(node_list))):
    adamic = {}
    for j in range(len(node_list)):
        if i != j:
            res = list(nx.jaccard_coefficient(G, [(node_list[i], node_list[j])]))[0]
            adamic[(res[0], res[1])] = res[2]
    relevant_nodes = list(sorted(adamic.items(), key=lambda item: item[1], reverse=True))[:10]
    for k in range(len(relevant_nodes)):
        attr_relevant_nodes[relevant_nodes[k][0][0]].append(relevant_nodes[k][0][1])

100%|█████████████████████████████████████████| 924/924 [00:40<00:00, 23.02it/s]


In [986]:
networkx.set_node_attributes(G, attr_relevant_nodes, 'relevant_nodes')

# Sentiment

In [1005]:
#df_companies = pd.read_csv('./tj_data/df_companies.csv')
df_info = pd.read_csv('./tj_data/df_info.csv')
#df_texts = pd.read_csv('./tj_data/df_texts.csv')
df_sentiment = pd.read_csv('./tj_data/tj_average_sentiment.csv')

In [991]:
#df_comp.to_csv('tj_comp_upd.csv', index=False)

In [1017]:
df_companies = pd.DataFrame(extracted_companies_, index=[0]).T.reset_index().rename({0: 'companies'}, axis=1)
df_companies['companies'] = df_companies['companies'].str.split(';')
df_companies = df_companies.explode('companies').rename({'index': 'doc_id'},axis=1)#.reset_index()

In [1018]:
df_companies

,doc_id,companies
0,0000000004,samsung
1,0000000015,apple
2,0000000018,apple
3,0000000020,apple
4,0000000031,google
...,...,...
55847,0000449990,twitter
55847,0000449990,facebook
55847,0000449990,instagram
55848,0000449991,instagram


In [1019]:
df_sentiment['post_id'] = df_sentiment['post_id'].astype(int)
df_companies['doc_id'] = df_companies['doc_id'].astype(int)


df_companies_sent = df_companies.merge(df_sentiment, how='left', left_on='doc_id', right_on='post_id')#.fillna(0)

In [1027]:
df_companies_sent[df_companies_sent['companies']=='aviator']

,doc_id,companies,post_id,neutral,negative,positive,skip,speech
82929,364655,aviator,NaN,NaN,NaN,NaN,NaN,NaN


In [1022]:
comp_sent = df_companies_sent.groupby('companies').agg({'neutral':'mean', 'negative': 'mean', 'positive': 'mean'}).reset_index()

In [1030]:
comp_sent = comp_sent[comp_sent['companies'].isin(set(node_list))].fillna(0)

In [1033]:
comp_sent.to_csv('tj_sent_upd.csv', index=False)

In [ ]:
neutr = {}
neg = {}
pos = {}
for i in range(len(comp_sent)):
    neutr[comp_sent.iloc[i]['companies']] = comp_sent.iloc[i]['neutral']
    neg[comp_sent.iloc[i]['companies']] = comp_sent.iloc[i]['negative']
    pos[comp_sent.iloc[i]['companies']] = comp_sent.iloc[i]['positive']

In [ ]:
networkx.set_node_attributes(G, neutr, 'neutral')
networkx.set_node_attributes(G, neg, 'negative')
networkx.set_node_attributes(G, pos, 'positive')

In [ ]:
#nx.write_gpickle(G, "G_tj_upd.gpickle")
#nx.write_gml(G, "G_tj_upd_upd.gml")

# Posts info

In [1032]:
info = {}
texts = {}
#extracted_companies_str = {}
#extracted_companies_keys = sorted(extracted_companies.keys())
for key in tqdm(files):
    with open(path + key) as json_file:
        data = json.load(json_file)
    info[key] = {}
    info[key]['post_id'] = data['result']['id']
    info[key]['auth_id'] = data['result']['author']['id']
    info[key]['commentsCount'] = data['result']['commentsCount']
    info[key]['hitsCount'] = data['result']['hitsCount']
    info[key]['likes'] = data['result']['likes']['count']
    info[key]['year'] = data['result']['dateRFC'].split()[3]
    #info[key]['tag'] = data['result']['subsite']['name']
    #extracted_companies_str[key] = ';'.join(extracted_companies[key])
    num_blocks = len(data['result']['blocks'])
    extracted_words_set = set()
    texts[key] = ''
    for i in range(num_blocks):
        if data['result']['blocks'][i]['type'] == 'text' or data['result']['blocks'][i]['type'] == 'header':
            texts[key] += data['result']['blocks'][i]['data']['text']
        texts[key] += ' '

100%|██████████████████████████████████| 298694/298694 [10:46<00:00, 461.72it/s]


In [1034]:
df_info = (pd.DataFrame.from_dict(info).T).reset_index().rename({'index': 'doc_id'}, axis=1)
#df_info.to_csv('tj_info_upd.csv', index=False)

In [1044]:
df_info['doc_id'] = df_info['doc_id'].astype(int)

In [1047]:
df_companies_info = df_companies.merge(df_info, how='left', on='doc_id').groupby('companies').agg({'commentsCount':'mean', 'hitsCount': 'mean', 'likes': 'mean'}).reset_index()#.to_csv('companies_likes.csv', index=False)

In [1049]:
df_companies_info = df_companies_info[df_companies_info['companies'].isin(set(node_list))].fillna(0)
df_companies_info.to_csv('tj_upd_likes.csv', index=False)

In [ ]:
df_companies_info

# Texts

In [1037]:
df_texts = (pd.DataFrame(texts, index=[0]).T).reset_index().rename({0:'text', 'index': 'doc_id'}, axis=1)

In [1058]:
def preprocessing_texts(df):
    df = df['text']
    df = re.sub(r'\(http[\S]*\)', '', df)
    df = re.sub(r'\n', '', df)
    df = re.sub(r'\[', '', df)
    df = re.sub(r'\]', '', df)
    return df

In [1059]:
df_texts['preprocessed'] = df_texts.apply(preprocessing_texts, axis=1)

In [1060]:
df_texts.to_csv('tj_texts_upd.csv', index=False)

In [1054]:
'''
tags = {}
for key in tqdm(files):
    with open(path + key) as json_file:
        data = json.load(json_file)
    tags[key] = {}
    tags[key]['tag'] = [item['text'] for item in data['result']['badges']]
'''

100%|██████████████████████████████████| 298694/298694 [09:57<00:00, 499.89it/s]


In [14]:
'''
tag_set = set()
for key in tags.keys():
    if len(tags[key]['tag'])!=0 and tags[key]['tag'][0] not in tag_set:
        print(tags[key])
        tag_set.add(tags[key]['tag'][0])
'''

"\ntag_set = set()\nfor key in tags.keys():\n    if len(tags[key]['tag'])!=0 and tags[key]['tag'][0] not in tag_set:\n        print(tags[key])\n        tag_set.add(tags[key]['tag'][0])\n"